In [1]:
import os
from openpyxl import load_workbook
import csv
from datetime import datetime, timedelta
import pandas as pd
from scipy.stats import chisquare
import math
import numpy as np

In [2]:
xlsx_filename = 'Semtex_assignment_heat_demand_forecast.xlsx'
tab_name = 'Input data'
csv_path = 'Semtex_Assignment.csv'
directory = 'Semtex_Assignment/0: Extract Data'
openpyxl_path = 'Semtex_Assignment_openpyxl.csv'

In [3]:
print("""
Extracting CSV from XLSX...
""")

# Load the Excel workbook
workbook = load_workbook(filename=xlsx_filename, data_only=True)

# Select the sheet to convert
sheet = workbook[tab_name]


print("""
WARNING: Package openpyxl changes '-' values to '0'. A manual copy has been made to keep the faulty values in the data.
WARNING: Manual save of CSV file does not compute millisecond range. Hard copy of range has been done to Hulife_Assignment.csv
""")

# Open a CSV file to write to
with open(openpyxl_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Iterate over the rows in the sheet and write them to the CSV file
    for row in sheet.iter_rows(min_row=1, max_col=sheet.max_column, max_row=sheet.max_row):
        writer.writerow([cell.internal_value if cell.internal_value is not None else '' for cell in row])



Extracting CSV from XLSX...


FileNotFoundError: [Errno 2] No such file or directory: '/home/drfeelgood/PycharmProjects/Hylife_Assignment/0: Extract Data/Hylife_assignment_heat_demand_forecast.xlsx'

In [ ]:
print("""
Making datetime uniform....
""")

def check_time_increments(file_path, time_column):
    # Read CSV file
    df = pd.read_csv(file_path)

    # Initialize a variable to hold the last time
    last_time = None

    # Loop over each row in the specified column
    for index, row in df.iterrows():
        # Determine the correct format string based on whether the datetime string includes fractional seconds
        date_str = row[time_column]
        
        # Identify the format of the date string
        if '.' in date_str:  # Fractional seconds are present
            if '-' in date_str:  # Format: YYYY-MM-DD HH:MM:SS.ffffff
                date_format = '%Y-%m-%d %H:%M:%S.%f'
            else:  # Format: MM/DD/YYYY HH:MM:SS.ffffff
                date_format = '%m/%d/%Y %H:%M:%S.%f'
        else:  # No fractional seconds
            if '-' in date_str:  # Format: YYYY-MM-DD HH:MM
                date_format = '%Y-%m-%d %H:%M'
            else:  # Format: MM/DD/YYYY HH:MM
                date_format = '%m/%d/%Y %H:%M'
        
        # Parse the string to a datetime object
        current_time = datetime.strptime(date_str, date_format)
        
        # Reformat the date string to the desired format
        desired_format = '%Y-%m-%d %H:%M:%S.%f'
        formatted_date_str = current_time.strftime(desired_format)
        
        # Update the DataFrame with the reformatted date string
        df.at[index, time_column] = formatted_date_str

        if last_time is not None:
            # Check if the current_time is less than last_time
            if current_time < last_time:
                print(f"Out-of-order date at index {index}: {last_time.strftime(desired_format)} to {formatted_date_str}")
            # Check if the current_time is not one hour ahead of the last_time
            elif current_time != last_time + timedelta(hours=1):
                print(f"Incorrect increment at index {index}: {last_time.strftime(desired_format)} to {formatted_date_str}")

        # Update the last_time to the current_time for the next iteration
        last_time = current_time

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)

# Replace with the path to your CSV file
time_column = "UTC"
check_time_increments(csv_path, time_column)


In [ ]:
print("""
Calculating differences between faulty time delta.
""")
def calculate_differences(date_object):
    # If the date is not at the beginning of the hour
    if date_object.minute != 0 or date_object.second != 0 or date_object.microsecond != 0:
        # Find the previous and next hour
        previous_hour = date_object.replace(minute=0, second=0, microsecond=0)
        next_hour = previous_hour + timedelta(hours=1)
        
        # Calculate the differences
        difference_from_previous = date_object - previous_hour
        difference_from_next = next_hour - date_object
        return difference_from_previous, difference_from_next
    return None, None


date_column_index = 0  # Adjust if the dates are in a different column
date_format = "%Y-%m-%d %H:%M:%S.%f"

with open(csv_path, 'r') as file:
    reader = csv.reader(file)
    
    # Skip header
    next(reader)
    
    for row_num, row in enumerate(reader, start=2):  # Starting from 2 to account for the header row
        date_str = row[date_column_index]
        try:
            date_object = datetime.strptime(date_str, date_format)
            diff_previous, diff_next = calculate_differences(date_object)
            if diff_previous and diff_next:
                print(f"Row {row_num}: {date_object} is {diff_previous} away from the previous hour and {diff_next} away from the next hour.")
        except ValueError:
            print(f"Invalid date format at row {row_num}: {date_str}")


A change in the timestamp fractional seconds has been noted, starting from the `2022-01-01 01:59:59.990000` timestamp. The incremental difference is `0.005` seconds per hour, ultimately stabilizing at `0.150` seconds from the `2022-01-02 05:59:59.850000` timestamp onward.


IF time was measured correctly, then there is a possibility for data to be set to seconds since epoch, allowing us to correct the faulty time intervals. This approach is not desired however, since we're talking about centiseconds on the hour and the project does not scale up (it's not a single space calculation done for many spaces).


### Possible Causes:

1. **Hardware Oscillator Inaccuracy:**
   The internal oscillator of the IoT device might not be accurate, leading to a drift in the device's internal clock. Hardware oscillators have a tolerance level, usually measured in parts per million (ppm), which defines how much the oscillator can deviate from the accurate time.

2. **Temperature Fluctuation:**
   Since the device is a temperature IoT device, it’s susceptible to environmental conditions. Changes in temperature can impact the internal oscillator’s speed, causing the clock to drift.

3. **Software Bug:**
   There might be a bug in the software running on the IoT device that is causing this behavior. This could be related to incorrect handling of leap seconds or other time synchronization logic.

4. **Time Synchronization Issues:**
   If the device is synchronizing its time with an external time source and there is an issue or error in the synchronization process, it might lead to anomalies in the timestamp.

5. **Manual Adjustment:**
   The device’s time might be getting adjusted manually, either by a user or by some automated process, leading to such shifts in time.

### Troubleshooting Steps:

- **Time Synchronization:**
  Ensure that the device’s clock is synchronized with a reliable time source, such as an NTP server, to correct for any drift.

- **Environmental Conditions:**
  Investigate the environmental conditions, like temperature and humidity, where the device is located to see if they are causing the drift.

- **Software and Firmware Review:**
  Examine the device's firmware or software for any potential bugs related to timekeeping or synchronization, and apply updates if available.

- **Hardware Specifications Review:**
  Review the specifications of the hardware, especially focusing on the oscillator's accuracy and tolerance, to understand if the hardware is the source of the drift.

### Conclusion:

Addressing the clock drift in IoT devices is crucial to maintain the accuracy and reliability of the timestamp data. Identifying the root cause, whether it's hardware inaccuracy, environmental conditions, software bugs, or synchronization issues, is the first step in resolving the time drift problem.


In [ ]:
print("""
Fixing faulty time to nearest hour...
""")
def round_to_nearest_hour(dt):
    if dt.minute == 0 and dt.second == 0 and dt.microsecond == 0:
        return dt
    rounded_dt = dt.replace(minute=0, second=0, microsecond=0)
    if dt.minute >= 30:
        rounded_dt += timedelta(hours=1)
    return rounded_dt

def round_datetimes_in_csv(file_path, datetime_column_index):
    with open(file_path, mode='r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    # Modify the datetime column
    for i, row in enumerate(rows):
        if i == 0:  # skip header row
            continue
        dt_str = row[datetime_column_index]
        dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S.%f')
        rounded_dt = round_to_nearest_hour(dt)
        row[datetime_column_index] = rounded_dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Write the modified rows back to the same file
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

# Usage example
round_datetimes_in_csv(csv_path, 0)  # Change 0 to the index of the datetime column in your CSV


In [ ]:
def update_csv_file(file_path):
    # Read the csv file into a DataFrame
    df = pd.read_csv(file_path, parse_dates=['UTC'])
    
    # Iterate over the first 50 rows and update the 'UTC' column where needed
    for index in range(min(50, len(df))):
        if df.loc[index, 'UTC'].year == 2022:
            # Update the year to 2021
            updated_date = df.loc[index, 'UTC'].replace(year=2021)
            df.at[index, 'UTC'] = updated_date
    
    # Write the updated DataFrame back to the csv file
    df.to_csv(file_path, index=False, date_format='%Y-%m-%d %H:%M:%S')

# Replace csv_path with the path to your csv file

update_csv_file(csv_path)


In [ ]:
print("""
Splitting CSV into its parts...
""")

def split_csv(file_path, save_directory):
    # Load the CSV file
    data = pd.read_csv(file_path)

    # Define the output file names
    file1_name = 'Historic_Data.csv'
    file2_name = 'Heat_Demand.csv'
    file3_name = 'Forecast_Data.csv'

    # Create full paths for the output files
    file1_path = os.path.join(save_directory, file1_name)
    file2_path = os.path.join(save_directory, file2_name)
    file3_path = os.path.join(save_directory, file3_name)

    # Find indices where all heat demand columns are not NULL
    not_null_heat_demand_indices = data[~data[['S31 Heat demand (%)', 'S32 Heat demand (%)', 'S41 Heat demand (%)']].isnull().any(axis=1)].index

    # Find indices where all heat demand columns are NULL but kWh is not NULL
    null_heat_demand_not_null_kwh_indices = data[
        data[['S31 Heat demand (%)', 'S32 Heat demand (%)', 'S41 Heat demand (%)']].isnull().any(axis=1) &
        ~data['Building Semtex OFFICE (kWh)'].isnull()
    ].index

    # Find indices where both heat demand columns and kWh are NULL
    null_heat_demand_and_kwh_indices = data[
        data[['S31 Heat demand (%)', 'S32 Heat demand (%)', 'S41 Heat demand (%)', 'Building Semtex OFFICE (kWh)']].isnull().any(axis=1)
    ].index

    # Check if any non-NULL values appear after NULL values in the respective columns
    if not null_heat_demand_not_null_kwh_indices.empty and max(null_heat_demand_not_null_kwh_indices) < max(null_heat_demand_and_kwh_indices):
        print("Warning: non-NULL kWh values appear after NULL kWh values in the file.")
    if max(not_null_heat_demand_indices) > min(null_heat_demand_not_null_kwh_indices):
        print("Warning: non-NULL heat demand values appear after NULL heat demand values in the file.")

    # Save the corresponding data frames to new CSV files
    data.loc[not_null_heat_demand_indices].to_csv(file1_path, index=False)
    data.loc[null_heat_demand_not_null_kwh_indices].to_csv(file2_path, index=False)
    data.loc[null_heat_demand_and_kwh_indices].to_csv(file3_path, index=False)

    print(f"{file1_path} has been created with indices {min(not_null_heat_demand_indices)+1} to {max(not_null_heat_demand_indices)+1}")
    print(f"{file2_path} has been created with indices {min(null_heat_demand_not_null_kwh_indices)+1} to {max(null_heat_demand_not_null_kwh_indices)+1}")
    print(f"{file3_path} has been created with indices {min(null_heat_demand_and_kwh_indices)+1} to {max(null_heat_demand_and_kwh_indices)+1}")

split_csv(csv_path, directory)


In [ ]:


from os import path

"""Analysis of Unusual Values"""

# Define the path of the historic data file
historic_data_file = path.join(directory, 'Historic_Data.csv')
forecast_data_file = path.join(directory, 'Forecast_Data.csv')
heat_demand_data_file = path.join(directory, 'Heat_Demand.csv')

# Read the historic data file
historic_data = pd.read_csv(historic_data_file)
forecast_data = pd.read_csv(forecast_data_file)
heat_demand_data = pd.read_csv(heat_demand_data_file)

In [ ]:
print("""
Checking for any faulty data, example "object" instead of "float64"....
""")
historic_data.info()
forecast_data.info()
heat_demand_data.info()

In [ ]:
def find_unusual_values(df, excluded_columns=None):
    """
    This function iterates over each cell in the input DataFrame to find unusual values
    and returns a DataFrame containing these unusual values with their column names and indices.
    
    :param df: DataFrame, the input data frame to be checked for unusual values.
    :param excluded_columns: list, optional, a list of columns to exclude from the check.
    :return: DataFrame, a data frame containing the unusual values.
    """

    if excluded_columns is None:
        excluded_columns = ["UTC"]

    unusual_values = {'column': [], 'index': [], 'value': []}

    # Drop columns where all values are NaN
    df = df.dropna(axis=1, how='all')

    for col in df.columns:
        if col not in excluded_columns:  # Skip excluded columns
            for idx, value in df[col].items():
                if pd.isna(value):  # Skip NaN values
                    continue
                try:
                    # Try to convert value to float
                    float(value)
                except ValueError:
                    # If conversion fails, it's an unusual value
                    unusual_values['column'].append(col)
                    unusual_values['index'].append(idx)
                    unusual_values['value'].append(value)

    # Convert the results to a DataFrame
    unusual_values_df = pd.DataFrame(unusual_values)
    return unusual_values_df



# Convert the results to a DataFrame
unusual_values_historic_df = find_unusual_values(historic_data)
unusual_values_forecast_df = find_unusual_values(forecast_data)
unusual_values_heat_demand_df = find_unusual_values(heat_demand_data)

In [ ]:
"""Function for extracting unique wrong values."""


def extract_unique_unusual_values(unusual_values_df, csv_file):
    """
    This function extracts unique unusual values from the provided DataFrame
    and returns a list of tuples containing the CSV file name, column name, and unique unusual values.
    
    :param unusual_values_df: DataFrame, the input data frame containing unusual values.
    :param csv_file: str, the name of the original CSV file from which the DataFrame was created.
    :return: list, a list of tuples containing the CSV file name, column name, and unique unusual values.
    """

    unique_unusual_values_list = []
    for column in unusual_values_df['column'].unique():
        # Filter DataFrame for each unique column
        column_df = unusual_values_df[unusual_values_df['column'] == column]

        # Extract unique unusual values for the column
        unique_unusual_values = column_df['value'].unique()
        # Append results to the list
        unique_unusual_values_list.append(csv_file)
        unique_unusual_values_list.append(column)
        unique_unusual_values_list.append(unique_unusual_values)

    return unique_unusual_values_list

In [ ]:
"""Historic_Data.csv: Contains unusual values - in the Outside temp (0.1 °C) column."""
historic_faulty_values = extract_unique_unusual_values(unusual_values_historic_df, historic_data_file)
historic_faulty_values

In [ ]:
"""Forecast_Data.csv: Contains NaN values in the columns S31 Heat demand (%), S32 Heat demand (%), S41 Heat demand (%), and Building Semtex OFFICE (kWh)."""
forecast_faulty_values = extract_unique_unusual_values(unusual_values_forecast_df, forecast_data_file)
forecast_faulty_values

In [ ]:
"""Heat_Demand.csv: Contains NaN values in the columns S31 Heat demand (%), S32 Heat demand (%), and S41 Heat demand (%)."""
heat_demand_faulty_values = extract_unique_unusual_values(unusual_values_heat_demand_df, heat_demand_data_file)
heat_demand_faulty_values

In [ ]:
data_ = historic_faulty_values + forecast_faulty_values + heat_demand_faulty_values
data_

In [ ]:
def analyze_unusual_values(data_list):
    results = {}

    for i in range(0, len(data_list), 3):
        csv_file = data_list[i]
        column_name = data_list[i + 1]
        unusual_value = data_list[i + 2]

        # Read the data file
        data = pd.read_csv(csv_file)

        # Extract rows with unusual values in the data
        unusual_rows = data.loc[data[column_name].isin(unusual_value)].copy()

        # Temporal Clustering Analysis
        unusual_rows['UTC'] = pd.to_datetime(unusual_rows['UTC'])
        time_diff = unusual_rows['UTC'].diff().dt.total_seconds() / 3600  # in hours

        # Frequency Analysis
        total_entries = len(data)
        erroneous_entries = len(unusual_rows)
        valid_entries = total_entries - erroneous_entries

        # Statistical Tests - Chi-square Test
        observed_frequencies = [erroneous_entries, valid_entries]
        expected_frequencies = [total_entries / 2, total_entries / 2]  # Assuming uniform distribution for the test
        chi_square_stat, p_value = chisquare(observed_frequencies, expected_frequencies)

        # Entropy Analysis
        p_erroneous = erroneous_entries / total_entries
        p_valid = valid_entries / total_entries

        # Check for log(0) case
        if erroneous_entries == 0 or valid_entries == 0:
            entropy = 0
        else:
            entropy = -(p_erroneous * math.log2(p_erroneous) + p_valid * math.log2(p_valid))

        # Date-Time Analysis
        during_work_hours = unusual_rows[(unusual_rows['UTC'].dt.weekday < 5) & (unusual_rows['UTC'].dt.hour >= 9) & (
                    unusual_rows['UTC'].dt.hour < 17)]
        outside_work_hours = unusual_rows[(unusual_rows['UTC'].dt.weekday < 5) & (
                    (unusual_rows['UTC'].dt.hour < 9) | (unusual_rows['UTC'].dt.hour >= 17))]
        weekends = unusual_rows[unusual_rows['UTC'].dt.weekday >= 5]

        # Adjust the categorization of the 9:00 and 17:00 timestamps
        during_work_hours = pd.concat([during_work_hours, unusual_rows[
            (unusual_rows['UTC'].dt.weekday < 5) & (unusual_rows['UTC'].dt.hour == 17)]])
        outside_work_hours = pd.concat([outside_work_hours, unusual_rows[
            (unusual_rows['UTC'].dt.weekday < 5) & (unusual_rows['UTC'].dt.hour == 9)]])

        date_time_analysis = {
            'During Work Hours': len(during_work_hours),
            'Outside Work Hours': len(outside_work_hours),
            'Weekends': len(weekends)
        }

        results[csv_file] = {
            'Column Name': column_name,
            'Unusual Value': unusual_value,
            'Frequency Analysis': {
                'Total Entries': total_entries,
                'Erroneous Entries': erroneous_entries,
                'Valid Entries': valid_entries
            },
            'Clustering Analysis': time_diff.describe().to_dict(),
            'Statistical Tests': {
                'Chi Square Statistic': chi_square_stat,
                'P Value': p_value
            },
            'Entropy Analysis': {
                'Entropy': entropy
            },
            'Date-Time Analysis': date_time_analysis
        }

    return results


In [ ]:
analysis_results = analyze_unusual_values(data_)

# Display the results in a formatted manner
for csv_file, result in analysis_results.items():
    print(f"CSV File: {csv_file}")
    print(f"Column Name: {result['Column Name']}")
    print(f"Unusual Value: {result['Unusual Value']}")
    print("Frequency Analysis:", result['Frequency Analysis'])
    print("Clustering Analysis:", result['Clustering Analysis'])
    print("Statistical Tests:", result['Statistical Tests'])
    print("Entropy Analysis:", result['Entropy Analysis'])
    print("Date-Time Analysis:", result['Date-Time Analysis'])  # New line to print Date-Time Analysis results
    print("---------------------------------------------------")

### Analysis of Unusual Values in Historic Data

- **Frequency Analysis:**
   - Total Entries: 8760
   - Erroneous Entries: 20
   - Valid Entries: 8740
   - Conclusion: A very small proportion of the dataset has erroneous entries.

- **Temporal Clustering Analysis:**
   - Mean Time Difference: 442.16 hours
   - Standard Deviation: 1194.46 hours
   - Minimum Time Difference: 1 hour
   - Maximum Time Difference: 5327 hours
   - Conclusion: Erroneous entries are spread out across the dataset without significant clustering.

- **Statistical Tests:**
   - Chi Square Statistic: 8680.18
   - P Value: 0.0
   - Conclusion: The occurrence of erroneous entries significantly differs from a uniform distribution, suggesting a non-random occurrence.

- **Entropy Analysis:**
   - Entropy: 0.0233
   - Conclusion: The dataset is highly ordered, and the occurrence of erroneous entries is not completely random.

- **Date-Time Analysis:**
   - During Work Hours: 0
   - Outside Work Hours: 12
   - Weekends: 8
   - Conclusion: Erroneous entries tend to occur outside work hours and on weekends, indicating a possible pattern related to operational hours.

**Overall Conclusion:** The erroneous values are few and are not uniformly distributed. They tend to appear outside work hours and on weekends, indicating a higher degree of order in their occurrences.


### Date-Time Analysis on Erroneous Entries in Historic Data

#### **CSV File:**
`/0: Extract Data/Historic_Data.csv`

#### **Column Name:**
`Outside temp (0.1 °C)`

#### **Unusual Value:**
`[' -   ']`

### **Analysis Results:**
#### **Frequency Analysis:**
- Total Entries: 8760
- Erroneous Entries: 20
- Valid Entries: 8740

#### **Clustering Analysis:**
- Count: 19.0
- Mean: 442.16 hours
- Std: 1194.46 hours
- Min: 1.0 hour with a single continuous value of 2 hours.
- 25%: 23.0 hours
- 50%: 133.0 hours
- 75%: 316.0 hours
- Max: 5327.0 hours

#### **Statistical Tests:**
- Chi Square Statistic: 8680.18
- P Value: 0.0

#### **Entropy Analysis:**
- Entropy: 0.0233

#### **Date-Time Analysis:**
- During Work Hours: 0
- Outside Work Hours: 12
- Weekends: 8

### **Interpretation:**
The Date-Time Analysis reveals that all the erroneous entries, represented by `-`, occur either outside typical work hours on weekdays or during weekends. Specifically:
- No erroneous entries are recorded during the conventional work hours (9 AM to 5 PM on weekdays).
- 12 erroneous entries are noted outside the standard work hours on weekdays.
- 8 erroneous entries are registered on weekends.

### **Conclusion:**
The exclusive presence of erroneous entries during non-operational hours and their absence during operational hours suggest several possibilities:
1. **Maintenance or Updates:** The system could be undergoing scheduled maintenance or updates during non-work hours, leading to disruptions in recording data.
2. **Connection Loss or Disruptions:** The occurrences of erroneous entries during non-operational hours might be indicative of a loss of connection or other disruptions, potentially due to reduced monitoring or operational levels during these times.
3. **Power Loss or Depleted Battery:** If the sensing device relies on a power source that is turned off during non-operational hours and weekends, it could be indicative of a depleted battery or power loss scenario.

Further investigation involving system logs, maintenance records, or other pertinent information sources would be essential to pinpoint the specific cause of these erroneous entries.


In [ ]:
analysis_results

In [ ]:
def clean_and_save_data(file_path, output_file_path):
    # Load the dataset
    historic_data = pd.read_csv(file_path)
    
    # Preprocessing: Replace '-' with NaN and convert column to numeric
    historic_data.loc[historic_data['Outside temp (0.1 °C)'] == '-', 'Outside temp (0.1 °C)'] = float('nan')
    historic_data['Outside temp (0.1 °C)'] = pd.to_numeric(historic_data['Outside temp (0.1 °C)'], errors='coerce')
    
    # Convert 'UTC' column to datetime
    historic_data['UTC'] = pd.to_datetime(historic_data['UTC'])
    
    # Save cleaned data to a new CSV file
    historic_data.to_csv(output_file_path, index=False)

# Usage
input_file_path = 'Historic_Data.csv'
output_file_path = 'Cleaned_Historic_Data.csv'

clean_and_save_data(input_file_path, output_file_path)
